# Gigaam

### Installing reqs and downloading examples

In [2]:
# If package is not installed
! pip install git+https://github.com/salute-developers/GigaAM.git

  Cloning https://github.com/salute-developers/GigaAM.git to /tmp/pip-req-build-_deucscs
  Running command git clone --filter=blob:none --quiet https://github.com/salute-developers/GigaAM.git /tmp/pip-req-build-_deucscs
  Resolved https://github.com/salute-developers/GigaAM.git to commit 2dfdad6195286784b496e6ec8d08e4de3b07313b
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 

In [3]:
# Downloading wavs for examples
!wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/example.wav
!wget https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/long_example.wav

--2025-05-24 05:52:58--  https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/example.wav
Resolving cdn.chatwm.opensmodel.sberdevices.ru (cdn.chatwm.opensmodel.sberdevices.ru)... 151.236.71.248
Connecting to cdn.chatwm.opensmodel.sberdevices.ru (cdn.chatwm.opensmodel.sberdevices.ru)|151.236.71.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 361324 (353K) [audio/x-wav]
Saving to: ‘example.wav’

example.wav         100%[===================>] 352.86K   720KB/s    in 0.5s    

2025-05-24 05:53:00 (720 KB/s) - ‘example.wav’ saved [361324/361324]

--2025-05-24 05:53:00--  https://cdn.chatwm.opensmodel.sberdevices.ru/GigaAM/long_example.wav
Resolving cdn.chatwm.opensmodel.sberdevices.ru (cdn.chatwm.opensmodel.sberdevices.ru)... 151.236.71.248
Connecting to cdn.chatwm.opensmodel.sberdevices.ru (cdn.chatwm.opensmodel.sberdevices.ru)|151.236.71.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2280109 (2.2M) [audio/x-wav]
Saving to: ‘long_

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pydub
!apt install ffmpeg -y

In [ ]:
from pydub import AudioSegment

root_dir = '/content/drive/MyDrive/Курсовая работа_2025/озвучка_valid_adapt' # или озвучка_test_adapt

audio_exts = ['.mp3', '.m4a', '.ogg', '.flac', '.aac', '.wma']

for subdir, _, files in os.walk(root_dir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in audio_exts:
            src_path = os.path.join(subdir, file)
            dst_path = os.path.splitext(src_path)[0] + '.wav'

            try:
                sound = AudioSegment.from_file(src_path)
                sound.export(dst_path, format="wav")
                os.remove(src_path)
                print(f"конвертировано и удалено: {src_path} → {dst_path}")
            except Exception as e:
                print(f"ошибка {src_path}: {e}")

## Speech Recognition

In [4]:
import os
import warnings
from typing import Dict

import gigaam

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
# model = gigaam.load_model(
#     "ctc",  # GigaAM-V2 CTC model
#     fp16_encoder=True,  # to use fp16 encoder weights - GPU only
#     use_flash=False,  # disable flash attention - colab does not support it
# )
# model.transcribe("sample.wav")

100%|███████████████████████████████████████| 444M/444M [00:22<00:00, 20.4MiB/s]


'какая погода в москве'

In [ ]:
# для test
from pathlib import Path
import re
from tqdm import tqdm

input_root = Path("/content/drive/MyDrive/Курсовая работа_2025/озвучка_test_adapt") # путь к файлу

output_root = Path("/content/drive/MyDrive/звук_gigaam") # путь к файлу
output_root.mkdir(parents=True, exist_ok=True)

model = gigaam.load_model("rnnt", device="cpu")

for voice_dir in input_root.iterdir():
    if voice_dir.is_dir():
        print(f"\nГруппа: {voice_dir.name}")
        all_lines = []

        # ищем .wav файлы
        wav_files = list(voice_dir.rglob("*.wav"))
        if not wav_files:
            print(f"нет .wav файлов в {voice_dir}")
            continue

        for wav_file in tqdm(wav_files, desc=f"{voice_dir.name}"):
            file_id = wav_file.stem

            try:
                text = model.transcribe(str(wav_file))
                if text.strip():
                    all_lines.append(f"# id: {file_id}\n{text.strip()}\n")
                else:
                    print(f"пусто: {file_id}")
            except Exception as e:
                print(f"ошибка в {wav_file.name}: {e}")

        if all_lines:
            output_path = output_root / f"{voice_dir.name}.conll"
            with output_path.open("w", encoding="utf-8") as out_f:
                out_f.write("\n".join(all_lines))
            print(output_path.name)
        else:
            print(f"нет данных для: {voice_dir.name}")

In [ ]:
# для valid
input_root = Path("/content/drive/MyDrive/Курсовая работа_2025/озвучка_valid_adapt")
output_root = Path("/content/drive/MyDrive/звук_gigaam")
output_root.mkdir(parents=True, exist_ok=True)

all_lines = []

wav_files = list(input_root.glob("*.wav"))
if not wav_files:
    print(f"нет .wav файлов в {input_root}")
else:
    for wav_file in tqdm(wav_files):
        file_id = wav_file.stem

        try:
            text = model.transcribe(str(wav_file))
            if text.strip():
                all_lines.append(f"# id: {file_id}\n{text.strip()}\n")
            else:
                print(f"пусто: {file_id}")
        except Exception as e:
            print(f"ошибка в {wav_file.name}: {e}")

    if all_lines:
        output_path = output_root / "valid_gigaam.conll"
        with output_path.open("w", encoding="utf-8") as out_f:
            out_f.write("\n".join(all_lines))
        print(f"\n сохранили: {output_path.name}")
    else:
        print(f"нет данных для: {voice_dir.name}")

### Long-Form Speech Recognition

In [ ]:
!pip install gigaam[longform]

* For long-form inference:
  * generate [Hugging Face API token](https://huggingface.co/docs/hub/security-tokens)
  * accept the conditions to access [pyannote/voice-activity-detection](https://huggingface.co/pyannote/voice-activity-detection) files and content
  * accept the conditions to access [pyannote/segmentation](https://huggingface.co/pyannote/segmentation) files and content

In [ ]:
os.environ["HF_TOKEN"] = "<HF_TOKEN>"

In [ ]:
model = gigaam.load_model("ctc", use_flash=False)
recognition_result = model.transcribe_longform("long_example.wav")

for utterance in recognition_result:
    transcription = utterance["transcription"]
    start, end = utterance["boundaries"]
    print(f"[{gigaam.format_time(start)} - {gigaam.format_time(end)}]: {transcription}")

[00:00:00 - 00:16:83]: вечерня отошла давно но в кельях тихо и темно уже и сам игумен строгий свои молитвы прекратил и кости ветхие склонил перекрестясь на одр убогий кругом и сон и тишина но церкви дверь отворена
[00:17:10 - 00:32:61]: трепещет луч лампады и тускло озаряет он и темную живопись икон и позлощенные оклады и раздается в тишине то тяжкий вздох то шепот важный и мрачно дремлет в вышине старинный свод
[00:32:95 - 00:49:33]: глухой и влажный стоят за клиросом чернец и грешник неподвижны оба и шепот их как глаз из гроба и грешник бледен как мертвец монах несчастный полно перестань
[00:49:82 - 01:05:74]: ужасна исповедь злодея заплачена тобою дань тому кто в злобе пламенея лукаво грешника блюдет и к вечной гибели ведет смирись опомнись время время раскаянье покров
[01:05:97 - 01:10:90]: я разрешу тебя грехов сложи мучительное бремя


## Emotion recognition

In [ ]:
model = gigaam.load_model("emo")
emotion2prob: Dict[str, int] = model.get_probs("example.wav")

print(", ".join([f"{emotion}: {prob:.3f}" for emotion, prob in emotion2prob.items()]))

100%|███████████████████████████████████████| 924M/924M [02:04<00:00, 7.78MiB/s]


angry: 0.000, sad: 0.002, neutral: 0.923, positive: 0.074


## GigaAM embeddings

In [ ]:
# audio-only pretrained encoder
model = gigaam.load_model("ssl", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

100%|███████████████████████████████████████| 887M/887M [03:21<00:00, 4.62MiB/s]


tensor([[[-0.2829,  0.3638,  0.4520,  ..., -0.4743, -0.4033, -0.2417],
         [ 0.1611, -0.5006, -0.0584,  ..., -0.6239, -0.2320, -0.2054],
         [-1.1849, -1.0029, -0.6111,  ..., -0.5137, -0.3737, -0.2654],
         ...,
         [ 0.0181, -0.3763, -0.8959,  ...,  0.1716,  0.0556,  0.1298],
         [ 0.2690, -0.0654, -0.5020,  ..., -1.4432, -1.4827, -1.4490],
         [-1.5650, -1.6693, -1.2834,  ...,  0.5117,  0.4839,  0.0136]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)


In [ ]:
# you also can embed audio with CTC- or RNNT-finetuned encoder
model = gigaam.load_model("ctc", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

tensor([[[-1.0334, -0.2841, -0.3606,  ..., -0.2859, -0.6947, -0.7006],
         [-0.4317, -0.0140, -0.9296,  ...,  0.1781,  0.2170, -0.0181],
         [-0.9221, -1.1284, -0.6389,  ..., -1.0664, -1.3304, -1.2421],
         ...,
         [ 0.5749,  0.5176, -0.0996,  ...,  1.7497,  1.8691,  2.1302],
         [-0.2919, -0.8087, -1.2554,  ..., -0.7942, -0.7634, -0.7938],
         [-1.8086, -2.1976, -2.4012,  ...,  0.8310,  1.0165,  1.0165]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)


## Export to ONNX

In [ ]:
onnx_dir = "onnx"
model_type = "rnnt" # or "ctc"

model = gigaam.load_model(
    model_type,
    fp16_encoder=False,  # only fp32 tensors
    use_flash=False,  # disable flash attention
)
model.to_onnx(dir_path=onnx_dir)

100%|███████████████████████████████████████| 892M/892M [03:20<00:00, 4.66MiB/s]


Succesfully ported onnx v2_rnnt_encoder to onnx/v2_rnnt_encoder.onnx.
Succesfully ported onnx v2_rnnt_decoder to onnx/v2_rnnt_decoder.onnx.
Succesfully ported onnx v2_rnnt_joint to onnx/v2_rnnt_joint.onnx.


### ONNX inference

In [ ]:
from gigaam.onnx_utils import load_onnx_sessions, transcribe_sample

sessions = load_onnx_sessions(onnx_dir, model_type)
transcribe_sample("example.wav", model_type, sessions)

'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

# VOSK


In [ ]:
!pip install vosk pydub
!apt-get install -y ffmpeg

In [ ]:
import wave
import json
from vosk import Model, KaldiRecognizer

In [ ]:
# путь к нужным папкам
input_dir = "/content/drive/MyDrive/Курсовая работа_2025/озвучка_valid_adapt"
output_root = "/content/drive/MyDrive/result_valid_adapt"
os.makedirs(output_root, exist_ok=True)

# загружаем модель
if not os.path.exists('vosk-model-small-ru-0.22'):
    !wget https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
    !unzip vosk-model-small-ru-0.22.zip
model = Model('vosk-model-small-ru-0.22')

# нужен WAV 16kHz mono для vosk
def convert_to_wav16k_mono(src_path, dst_path):
    try:
        audio = AudioSegment.from_file(src_path)
        audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
        audio.export(dst_path, format="wav")
        return True
    except Exception as e:
        print(f"ошибка {src_path}: {e}")
        return False

for root, _, files in os.walk(input_dir):
    for file in files:
        if file.lower().endswith(('.wav', '.mp3', '.m4a', '.flac', '.aac', '.ogg')):
            full_path = os.path.join(root, file)
            file_id = os.path.splitext(file)[0]


            relative_path = os.path.relpath(root, input_dir)
            output_folder = os.path.join(output_root, relative_path, file_id)
            os.makedirs(output_folder, exist_ok=True)

            temp_path = "/content/temp.wav"
            print(f"обработка: {file_id}")

            if not convert_to_wav16k_mono(full_path, temp_path):
                continue

            try:
                wf = wave.open(temp_path, 'rb')
                rec = KaldiRecognizer(model, wf.getframerate())

                results = []
                while True:
                    data = wf.readframes(4000)
                    if len(data) == 0:
                        break
                    if rec.AcceptWaveform(data):
                        res = json.loads(rec.Result())
                        results.append(res.get('text', ''))
                res = json.loads(rec.FinalResult())
                results.append(res.get('text', ''))
                final_text = ' '.join(results).strip()
                tokens = final_text.split()

                output_file = os.path.join(output_folder, f"{file_id}.conll")
                with open(output_file, "w", encoding="utf-8") as f:
                    f.write(f"# id: {file_id}\n")
                    f.write(f"# text RU: {final_text}\n")
                    for idx, token in enumerate(tokens, 1):
                        f.write(f"{idx}\t{token}\tO\n")

                print(f"сохранили: {output_file}")
            except Exception as e:
                print(f"ошибка {file_id}: {e}")

In [ ]:

input_root = Path("/content/drive/MyDrive/result_test_adapt")
output_root = Path("/content/drive/MyDrive/звук_vosk")
output_root.mkdir(parents=True, exist_ok=True)

for voice_dir in input_root.iterdir():
    if voice_dir.is_dir():
        print(f"\nГруппа: {voice_dir.name}")
        all_lines = []

        conll_files = list(voice_dir.rglob("*.conll"))
        if not conll_files:
            print(f".conll файлов в {voice_dir}")
            continue

        for file in conll_files:
            print(file.relative_to(input_root))
            try:
                with file.open(encoding="utf-8") as f:
                    lines = f.read().splitlines()
            except Exception as e:
                print(f"ошибка {file.name}: {e}")
                continue

            current_id = None
            current_text = None

            for line in lines:
                if line.startswith("# id:"):
                      match = re.search(r'# id:\s*(?:id\s*)?(.+)', line)
                      if match:
                          current_id = match.group(1).strip()
                elif line.startswith("# text RU:"):
                    current_text = line.split(":", 1)[-1].strip()

            if current_id and current_text:
                all_lines.append(f"# id: {current_id}\n{current_text}\n")
            else:
                print(f"пропуск {file.name}")

        if all_lines:
            output_path = output_root / f"{voice_dir.name}.conll"
            with output_path.open("w", encoding="utf-8") as out_f:
                out_f.write("\n".join(all_lines))
            print(f"сохранили {output_path.name}")
        else:
             print("ничего нет")

In [ ]:
input_root = Path("/content/drive/MyDrive/result_valid_adapt")
output_root = Path("/content/drive/MyDrive/звук_vosk")
output_root.mkdir(parents=True, exist_ok=True)

output_file = output_root / "valid_vosk.conll"

all_lines = []

conll_files = list(input_root.rglob("*.conll"))
if not conll_files:
    print(f".conll файлов в {input_root}")
else:
    for file in conll_files:
        print(file.relative_to(input_root))
        try:
            with file.open(encoding="utf-8") as f:
                lines = f.read().splitlines()
        except Exception as e:
            print(f"ошибка {file.name}: {e}")
            continue

        current_id = None
        current_text = None

        for line in lines:
            if line.startswith("# id:"):
                match = re.search(r'# id:\s*(?:id\s*)?(.+)', line)
                if match:
                    current_id = match.group(1).strip()
            elif line.startswith("# text RU:"):
                current_text = line.split(":", 1)[-1].strip()

        if current_id and current_text:
            all_lines.append(f"# id: {current_id}\n{current_text}\n")
        else:
            print(f"пропуск {file.name}")

if all_lines:
    with output_file.open("w", encoding="utf-8") as out_f:
        out_f.write("\n".join(all_lines))
    print(f"сохранили {output_file.name}")
else:
    print("ничего нет")

In [ ]:
import re
from pathlib import Path

folder = Path("/content/drive/MyDrive/звук_gigaam")

for file_path in folder.glob("*.conll"):
    print(file_path.name)
    try:
        with file_path.open("r", encoding="utf-8") as f:
            lines = f.readlines()
        cleaned_lines = []
        for line in lines:
            if line.startswith("# id:"):
                match = re.search(r'# id:\s*(?:id\s*)?(.+)', line)
                if match:
                    cleaned_line = f"# id: {match.group(1).strip()}\n"
                    cleaned_lines.append(cleaned_line)
                else:
                    cleaned_lines.append(line)
            else:
                cleaned_lines.append(line)

        with file_path.open("w", encoding="utf-8") as f:
            f.writelines(cleaned_lines)
        print(file_path.name)

    except Exception as e:
        print(f"ошибка {file_path.name}: {e}")

# WER, CER

In [ ]:
!pip install evaluate
!pip install jiwer

In [ ]:
from evaluate import load
from jiwer import cer
wer = load("wer")
wer_score = wer.compute(predictions=predictions, references=references)

In [ ]:
vosk_dir = "/content/drive/MyDrive/звук_gigaam" # vosk
reference_path = "/content/drive/MyDrive/Курсовая работа_2025/ru.valid_adapt.conll"
wer = load("wer")

def parse_predictions(file_path):
    preds = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        current_id = None
        for line in f:
            line = line.strip()
            if line.startswith('# id:'):
                current_id = line.split(':', 1)[1].strip()
            elif line and current_id:
                preds[current_id] = line
                current_id = None
    return preds
def parse_references(file_path):
    refs = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        current_id = None
        for line in f:
            line = line.strip()
            if line.startswith('# id:'):
                current_id = line.split(':', 1)[1].strip()
            elif line.startswith('# text RU:'):
                if current_id:
                    refs[current_id] = line.split(':', 1)[1].strip()
    return refs

#  WER для пары prediction + reference
def compute_wer(pred_file, ref_dict):
    preds = parse_predictions(pred_file)
    matched_preds = []
    matched_refs = []
    for ex_id in preds:
        if ex_id in ref_dict:
            matched_preds.append(preds[ex_id])
            matched_refs.append(ref_dict[ex_id])
    if not matched_preds:
        print(f"нет совпадений в {pred_file}")
        return None
    score = wer.compute(predictions=matched_preds, references=matched_refs)
    print(f"{os.path.basename(pred_file)}: WER = {score:.4f}")
    return score
ref_dict = parse_references(reference_path)
results = {}
for fname in os.listdir(vosk_dir):
        fpath = os.path.join(vosk_dir, fname)
        score = compute_wer(fpath, ref_dict)
        if score is not None:
            results[fname] = score

In [ ]:
vosk_dir = "/content/drive/MyDrive/звук_vosk"
reference_path = "/content/drive/MyDrive/Курсовая работа_2025/ru.test_adapt.conll"

def parse_predictions(file_path):
    preds = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        current_id = None
        for line in f:
            line = line.strip()
            if line.startswith('# id:'):
                current_id = line.split(':', 1)[1].strip()
            elif line and current_id:
                preds[current_id] = line
                current_id = None
    return preds
def parse_references(file_path):
    refs = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        current_id = None
        for line in f:
            line = line.strip()
            if line.startswith('# id:'):
                current_id = line.split(':', 1)[1].strip()
            elif line.startswith('# text RU:'):
                if current_id:
                    refs[current_id] = line.split(':', 1)[1].strip()
    return refs
# CER для пары prediction + reference
def compute_wer(pred_file, ref_dict):
    preds = parse_predictions(pred_file)
    matched_preds = []
    matched_refs = []
    for ex_id in preds:
        if ex_id in ref_dict:
            matched_preds.append(preds[ex_id])
            matched_refs.append(ref_dict[ex_id])
    if not matched_preds:
        print(f"нет совпадений в {pred_file}")
        return None
    score = cer(matched_refs, matched_preds)
    print(f"{os.path.basename(pred_file)}: CER = {score:.4f}")
    return score
ref_dict = parse_references(reference_path)
results = {}
for fname in os.listdir(vosk_dir):
        fpath = os.path.join(vosk_dir, fname)
        score = compute_wer(fpath, ref_dict)
        if score is not None:
            results[fname] = score